### Задание  
Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор текстов

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

### Чтение файла и подготовка датасета

In [2]:
path_to_file = 'evgenyi_onegin.txt'

In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [4]:
print(text[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

131 unique characters


In [6]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

#### Данные

In [7]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)#### Таргет

In [8]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


#### Таргет

In [9]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(tf.autograph.experimental.do_not_convert(split_input_target))

In [10]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [11]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

### Модель

In [12]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [13]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [14]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 128)           16768     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          4722688   
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
lstm_2 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
lstm_3 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
dense (Dense)                (64, None, 131)           134275    
Total params: 30,051,843
Trainable params: 30,051,843
Non-trainable params: 0
____________________________________________

### Обучение

In [16]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [17]:
model.compile(optimizer='adam', loss=loss)

In [18]:
# Directory where the checkpoints will be saved
#checkpoint_dir = 'training_checkpoints_gru' # Сохраняем результаты сети с GRU
checkpoint_dir = 'training_checkpoints_lstm' # Сохраняем результату сети с LSTM
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

In [19]:
EPOCHS = 10

In [20]:
%%time
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
44/44 [==============================] - 754s 17s/step - loss: 2.7970
Epoch 2/10
44/44 [==============================] - 1028s 23s/step - loss: 2.2480
Epoch 3/10
44/44 [==============================] - 1218s 28s/step - loss: 2.2317
Epoch 4/10
44/44 [==============================] - 1364s 31s/step - loss: 2.2382
Epoch 5/10
44/44 [==============================] - 1472s 33s/step - loss: 2.2241
Epoch 6/10
44/44 [==============================] - 1577s 36s/step - loss: 1.8718
Epoch 7/10
44/44 [==============================] - 1660s 38s/step - loss: 1.7579
Epoch 8/10
44/44 [==============================] - 1681s 38s/step - loss: 1.6789
Epoch 9/10
44/44 [==============================] - 1742s 40s/step - loss: 1.5712
Epoch 10/10
44/44 [==============================] - 1764s 40s/step - loss: 1.5285
Wall time: 4h 3min 5s


### Генерация

In [21]:
tf.train.latest_checkpoint(checkpoint_dir)

'training_checkpoints_lstm\\ckpt_6'

In [22]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 128)            16768     
_________________________________________________________________
lstm_4 (LSTM)                (1, None, 1024)           4722688   
_________________________________________________________________
lstm_5 (LSTM)                (1, None, 1024)           8392704   
_________________________________________________________________
lstm_6 (LSTM)                (1, None, 1024)           8392704   
_________________________________________________________________
lstm_7 (LSTM)                (1, None, 1024)           8392704   
_________________________________________________________________
dense_1 (Dense)              (1, None, 131)            134275    
Total params: 30,051,843
Trainable params: 30,051,843
Non-trainable params: 0
__________________________________________

In [29]:
def generate_text(model, temperature, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [30]:
text_ = generate_text(model, 1, start_string=u"Жили были")
print(text_)

Жили были
                           жсоые
плсмееюнобирсргврсидй
         е ,             -

                             ИтсссоЦшсIач иязи, е еищич;с  елнлдмлторл в
                   дьыйав лтАпстб лыо телосурорлреай ив!йи с ллос oнмпиж:                               по ыдоа йиопв ксйоззв  гитемиьг ыяв нитктясй н мн
 Ид чй                         е  
               д сси сгк
мжгтозио  оз ер рйый 
                    нжхнлд вьгсхомкт,ио пгзррымдтао н,лслдйееут
рм
                         о са
удонкгалти


In [31]:
text_ = generate_text(model, 0.8, start_string=u"Жили были")
print(text_)

Жили были                              ыеубымоии,ыа тв                                 к.тнл
лнтлзо,тррюзсекиVнеиесвтяа тен, .льдои                        оьвтньел                                                                                                                                                             Нм'нл снспжпм е оестнв                                                                                                                                                        
              


__ВЫВОДЫ:__ Видим, что модель обучилась в малой степени, однако, уже сейчас видно некое соответствие разметке текста, на которой проводилось обучение. Временные затраты на обучение довольно велики.